# Transferring Topo stuff to python

In [ ]:
# Import packages 
import os
import sys

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.tri as tri
import matplotlib.colors as colors
import matplotlib.gridspec as gsp

from scipy.io import FortranFile

import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import importlib
import glob
import copy

import Plots as Po

importlib.reload(Po)

#sys.path.append('../Plotting/')
#   Now you can imprt modules in ../Plotting
#import xyp_plot as xyp
#import ana as a


## Set some paths and filenames for topography files

In [ ]:
TopoRoot = '/project/amp/juliob/Topo-generate-devel/Topo/Topo.git/'
Case = 'c540_CONUS_06'
cgrid= 'geos_c540_CONUS'
Gridkey='c'

In [ ]:
TopoRoot = '/project/amp/juliob/Topo-generate-devel/Topo/Topo.git/'
Case = 'c1080_CONUS_06'
cgrid= 'geos_c1080_CONUS'
Gridkey='c'

In [ ]:
TopoRoot = '/project/amp/juliob/Topo-generate-devel/Topo/Topo.git/'
Case = 'ne30pg3_co60_fi0_bugfix'
cgrid= 'ne30pg3'
Gridkey='c'

In [ ]:
TopoRoot = '/project/amp/juliob/Topo-generate-devel/Topo/Topo.git/'
Case = 'fv_0.9x1.25_Sco100_fix'
cgrid= 'fv_0.9x1.25'
Gridkey='yx'

## Set longitude and latitude range for check

In [ ]:
#Some locations
Regions=['Colorado','Western_US','Southern_Andes','Assam']
Region=Regions[1]
print(Region)
terrlev=[-100,0,10,20,50,100,200,500,1000,1500,2000,2500,3000 ] #,3500,4000]
devlev=np.linspace(0,2000,num=21)

# Note panels are numbered 0,...,5
if (Region=='Colorado'):
    #Colorado/New Mexico
    wlon,elon= -110.,-105.
    slat,nlat=36.,40.
    ip=3

if (Region=='Western_US'):
    #Western US
    wlon,elon=-125.,-95.
    slat,nlat=15,45
    ip=3

if (Region=='Southern_Andes'):
    #Southern Andes
    wlon,elon=-77.,-57.
    slat,nlat=-58,-45
    ip=4

if (Region=='Assam'):
    wlon,elon=80.,110.
    slat,nlat=10.,40.
    ip=1
    terrlev=[-100,0,10,20,50,100,200,500,1000,1500,2000,2500,3000,3500,4000,4500,5000.,6000.,7000.,8000.]
    devlev=np.linspace(-1000,3000,num=21)

if (elon<0.):
    elon=elon+360.
if (wlon<0.):
    wlon=wlon+360.

lonrange=np.asarray([wlon,elon] ) #-2
latrange=np.asarray([slat,nlat]) #-4



In [ ]:
foo=glob.glob( TopoRoot +'/cases/'+ Case + '/output/'+cgrid+'*.nc')
FileN = foo[-1]
print(FileN)
dCAM=xr.open_dataset( FileN )
print( list(dCAM.variables) )
print(np.shape(dCAM['lat']))
lonc=dCAM['lon'].values
latc=dCAM['lat'].values
angl=dCAM['ANGLL'].values
hwdt=dCAM['HWDTH'].values
clng=dCAM['CLNGT'].values
mxd=dCAM['MXDIS'].values
sgh=dCAM['SGH'].values


In [ ]:
if (Gridkey=='c'):
    zoo=np.where( ( (lonc<=lonrange[1])&(lonc>lonrange[0]) )&( (latc<=latrange[1])&(latc>latrange[0]) ) )
    print(len(zoo[0]))
    iroo=zoo[0]
elif (Gridkey=='yx'):
    zoox=np.where( ( (lonc<=lonrange[1])&(lonc>lonrange[0]) ) )
    zooy=np.where( ( (latc<=latrange[1])&(latc>latrange[0]) ) )
    print(len(zoox[0]),len(zooy[0]))
    iroox=zoox[0]
    irooy=zooy[0]

print(np.shape(angl))

In [ ]:
foo=glob.glob( TopoRoot +'/cases/'+ Case + '/output/topo*.nc')
FileN = foo[-1]
print(FileN)
topo=xr.open_dataset( FileN )
print(list(topo.variables))
print(topo.attrs)
raw=topo['terr_dev']+topo['terr_sm']
Loo = np.size(raw)
nc=int(np.sqrt( Loo/6 ))
print(int(nc))
raw=np.reshape( raw.values , (6,nc,nc) )
dev=np.reshape( topo['terr_dev'].values , (6,nc,nc) )
smoo=np.reshape( topo['terr_sm'].values , (6,nc,nc) )
tlats=np.reshape( topo['lat'].values , (6,nc,nc) )
tlons=np.reshape( topo['lon'].values , (6,nc,nc) )


In [ ]:
# RemapFile = 'remap_nc0540_Nsw008_Nrs000_Co012_Fi000_vX_20230106_1205.dat'
# FileN=TopoRoot +'/cases/'+ Case + '/output/'+RemapFile

# Do a file search in output directory
# Pick last (most recent) file
foo=glob.glob( TopoRoot +'/cases/'+ Case + '/output/remap*dat')
FileN = foo[-1]

f=FortranFile( FileN, 'r')
idims=f.read_record( '<i4' )

nc=idims[0]

uniqi=f.read_record( '<f8'   ).reshape( 6,nc,nc)
aniso=f.read_record( '<f8'   ).reshape( 6,nc,nc)
anglx=f.read_record( '<f8'   ).reshape( 6,nc,nc)
mxdis=f.read_record( '<f8'   ).reshape( 6,nc,nc)
hwdth=f.read_record( '<f8'   ).reshape( 6,nc,nc)
clngt=f.read_record( '<f8'   ).reshape( 6,nc,nc)
block=f.read_record( '<f8'   ).reshape( 6,nc,nc)
profi=f.read_record( '<f8'   ).reshape( 6,nc,nc)
nodes=f.read_record( '<f8'   ).reshape( 6,nc,nc)
wedge=f.read_record( '<f8'   ).reshape( 6,nc,nc)
nodos=f.read_record( '<f8'   ).reshape( 6,nc,nc)
wedgo=f.read_record( '<f8'   ).reshape( 6,nc,nc)



In [ ]:
terrlev=[-100,0,10,20,50,100,200,500,1000,2000,3000,4000,5000]
terrlev=np.linspace(-100,4000,42)
plt.figure( figsize=(18,15))
plt.contourf( tlons[ip,:,:],tlats[ip,:,:],mxdis[ip,:,:], levels=np.linspace(-500,2000,num=26), cmap='terrain' ,alpha=.8) #,
#plt.contourf( tlons[ip,:,:],tlats[ip,:,:],wedgo[ip,:,:], levels=np.linspace(-500.,2000.,num=26), cmap='terrain' ,alpha=.8) #,
#plt.contourf( tlons[ip,:,:],tlats[ip,:,:],dev[ip,:,:], levels=np.linspace(-500,2000.,num=21), cmap='terrain' ,alpha=.2) #,
#plt.contourf( tlons[ip,:,:],tlats[ip,:,:],raw[ip,:,:], levels=terrlev, cmap='terrain',alpha=1.) #,
plt.xlim(lonrange)
plt.ylim(latrange)


#plt.plot( [40,90],[0,30])
if (Gridkey=='c'):
    for ir in iroo:
        lat0,lon0 = latc[ir],lonc[ir]
        lat1,lon1,lat2,lon2 = Po.CAMridgelet(lat=lat0 ,lon=lon0 , angll=angl[0,ir], clngt=clng[0,ir] )
        plt.plot( [lon1,lon2] , [lat1,lat2], color='black' )
if (Gridkey=='yx'):
    for iry in irooy:
        for irx in iroox:
            lat0,lon0 = latc[iry],lonc[irx]
            lat1,lon1,lat2,lon2 = Po.CAMridgelet(lat=lat0 ,lon=lon0 , angll=angl[0,iry,irx], clngt=clng[0,iry,irx] )
            if(mxd[0,iry,irx]>100.):
                plt.plot( [lon1,lon2] , [lat1,lat2], color='black' )


#ax1 = fig.add_subplot(3, 4, ipoo , projection=ccrs.PlateCarree(central_longitude=0))
#cf = ax1.contourf(xlon, xlat, gutnx[L,:,:],  levels=clevs, cmap=cmap  , transform=ccrs.PlateCarree())

In [ ]:
terrlev=[-100,0,10,20,50,100,200,500,1000,1500,2000,2500,3000,3500,4000]
devlev=np.linspace(0,2000,num=21)
cmap=plt.cm.terrain
fig,ax = plt.subplots(2,2,figsize=(15,15) ,squeeze=False ) # , sharex=True)


j,i=0,0
#plt.contourf( tlons[ip,:,:],tlats[ip,:,:],mxdis[ip,:,:], levels=np.linspace(-100,500.,num=21), cmap='terrain' ,alpha=.5) #,
ax[j,i].contourf( tlons[ip,:,:],tlats[ip,:,:],raw[ip,:,:], levels=terrlev, cmap='terrain',alpha=.5) #,
ax[j,i].set_xlim(lonrange)
ax[j,i].set_ylim(latrange)

j,i=0,1
#plt.contourf( tlons[ip,:,:],tlats[ip,:,:],mxdis[ip,:,:], levels=np.linspace(-100,500.,num=21), cmap='terrain' ,alpha=.5) #,
ax[j,i].contourf( tlons[ip,:,:],tlats[ip,:,:],smoo[ip,:,:], levels=terrlev, cmap='terrain',alpha=.5) #,
ax[j,i].set_xlim(lonrange)
ax[j,i].set_ylim(latrange)

j,i=1,0
#plt.contourf( tlons[ip,:,:],tlats[ip,:,:],mxdis[ip,:,:], levels=np.linspace(-100,500.,num=21), cmap='terrain' ,alpha=.5) #,
ax[j,i].contourf( tlons[ip,:,:],tlats[ip,:,:],dev[ip,:,:], levels=devlev, cmap='terrain',alpha=1) #,
ax[j,i].set_xlim(lonrange)
ax[j,i].set_ylim(latrange)

j,i=1,1
if (Gridkey=='c'):
    ax[j,i].tricontourf(lonc,latc,mxd[0,:], levels=devlev, cmap='terrain',alpha=1)
    for ir in iroo:
        lat0,lon0 = latc[ir],lonc[ir]
        lat1,lon1,lat2,lon2 = Po.CAMridgelet(lat=lat0 ,lon=lon0 , angll=angl[0,ir], clngt=clng[0,ir] )
        plt.plot( [lon1,lon2] , [lat1,lat2], color='black' )
    
    
if (Gridkey=='yx'):
    ax[j,i].contourf(lonc,latc,mxd[0,:,:]-1, levels=devlev, cmap='terrain',alpha=1)
    for iry in irooy:
        for irx in iroox:
            lat0,lon0 = latc[iry],lonc[irx]
            lat1,lon1,lat2,lon2 = Po.CAMridgelet(lat=lat0 ,lon=lon0 , angll=angl[0,iry,irx], clngt=clng[0,iry,irx] )
            if(mxd[0,iry,irx]>100.):
                plt.plot( [lon1,lon2] , [lat1,lat2], color='black' )
    
    
ax[j,i].set_xlim(lonrange)
ax[j,i].set_ylim(latrange)


#ax1 = fig.add_subplot(3, 4, ipoo , projection=ccrs.PlateCarree(central_longitude=0))
#cf = ax1.contourf(xlon, xlat, gutnx[L,:,:],  levels=clevs, cmap=cmap  , transform=ccrs.PlateCarree())

In [ ]:
cmap=plt.cm.terrain

"""
fig,ax = plt.subplots(2,3,figsize=(21,20) ,squeeze=False ) # , sharex=True)

"""
fig = plt.figure(figsize=(21,20)  ) # , sharex=True)
gs = gsp.GridSpec( 2, 3, figure=fig ,wspace=0.1, hspace=0.1 )
    
j,i=0,0
ax=fig.add_subplot(gs[j,i])
u=ax.contourf( tlons[ip,:,:],tlats[ip,:,:],raw[ip,:,:], levels=terrlev, cmap='terrain',alpha=.5) #,
ax.set_xlim(lonrange)
ax.set_ylim(latrange)
plt.colorbar(u,ax=ax,location='bottom',shrink=.9)
ax.set_title(loc='left',label='a) Raw 3km topography' , fontsize=14)


j,i=0,1
#plt.contourf( tlons[ip,:,:],tlats[ip,:,:],mxdis[ip,:,:], levels=np.linspace(-100,500.,num=21), cmap='terrain' ,alpha=.5) #,
ax=fig.add_subplot(gs[j,i])
u=ax.contourf( tlons[ip,:,:],tlats[ip,:,:],smoo[ip,:,:], levels=terrlev, cmap='terrain',alpha=.5) #,
ax.set_xlim(lonrange)
ax.set_ylim(latrange)
plt.colorbar(u,ax=ax,location='bottom',shrink=.9)
ax.set_title(loc='left',label='b) 3km topography smoothed with a \n radial scale $\sim$ 180km ' , fontsize=14)

j,i=1,0
#plt.contourf( tlons[ip,:,:],tlats[ip,:,:],mxdis[ip,:,:], levels=np.linspace(-100,500.,num=21), cmap='terrain' ,alpha=.5) #,
ax=fig.add_subplot(gs[j,i])
u=ax.contourf( tlons[ip,:,:],tlats[ip,:,:],dev[ip,:,:], levels=devlev, cmap='terrain',alpha=1) #,
ax.set_xlim(lonrange)
ax.set_ylim(latrange)
plt.colorbar(u,ax=ax,location='bottom',shrink=.9)

j,i=1,1
ax=fig.add_subplot(gs[j,i])
if (Gridkey=='c'):
    u=ax.tricontourf(lonc,latc,mxd[0,:], levels=devlev, cmap='terrain',alpha=1)
    for ir in iroo:
        lat0,lon0 = latc[ir],lonc[ir]
        lat1,lon1,lat2,lon2 = Po.CAMridgelet(lat=lat0 ,lon=lon0 , angll=angl[0,ir], clngt=clng[0,ir] )
        plt.plot( [lon1,lon2] , [lat1,lat2], color='black' )
plt.colorbar(u,ax=ax,location='bottom',shrink=.9)
    
    
if (Gridkey=='yx'):
    u=ax.contourf(lonc,latc,mxd[0,:,:]-1, levels=devlev, cmap='terrain',alpha=1)
    for iry in irooy:
        for irx in iroox:
            lat0,lon0 = latc[iry],lonc[irx]
            lat1,lon1,lat2,lon2 = Po.CAMridgelet(lat=lat0 ,lon=lon0 , angll=angl[0,iry,irx], clngt=clng[0,iry,irx] )
            if(mxd[0,iry,irx]>100.):
                ax.plot( [lon1,lon2] , [lat1,lat2], color='black' )
    
ax.set_xlim(lonrange)
ax.set_ylim(latrange)

"""
j,i=1,2
#plt.contourf( tlons[ip,:,:],tlats[ip,:,:],mxdis[ip,:,:], levels=np.linspace(-100,500.,num=21), cmap='terrain' ,alpha=.5) #,
ax=fig.add_subplot(gs[j,i])
u=ax.contourf( tlons[ip,:,:],tlats[ip,:,:],wedgo[ip,:,:], levels=devlev, cmap='terrain',alpha=1) #,
ax.set_xlim(lonrange)
ax.set_ylim(latrange)
plt.colorbar(u,ax=ax,location='bottom',shrink=.9)
"""

j,i=1,2
ax=fig.add_subplot(gs[j,i])
if (Gridkey=='c'):
    u=ax.tricontourf(lonc,latc,sgh[:]-1, levels=devlev, cmap='terrain',alpha=1)
if (Gridkey=='yx'):
    u=ax.contourf(lonc,latc,sgh[:,:]-1, levels=devlev, cmap='terrain',alpha=1)
ax.set_xlim(lonrange)
ax.set_ylim(latrange)
plt.colorbar(u,ax=ax,location='bottom',shrink=.9)


#ax1 = fig.add_subplot(3, 4, ipoo , projection=ccrs.PlateCarree(central_longitude=0))
#cf = ax1.contourf(xlon, xlat, gutnx[L,:,:],  levels=clevs, cmap=cmap  , transform=ccrs.PlateCarree())

In [ ]:
plt.figure( figsize=(18,15))
#plt.contourf( tlons[ip,:,:],tlats[ip,:,:],mxdis[ip,:,:], levels=np.linspace(-100,500.,num=21), cmap='terrain' ,alpha=.5) #,
plt.contourf(dev[ip,:,:], levels=np.linspace(-100,100.,num=21), cmap='terrain',alpha=.5) #,
plt.xlim(0,1000)
plt.ylim(1500,2500)
plt.colorbar()